# TABLE OF CONTENTS:
---
* [Notebook Summary](#Notebook-Summary)
* [Setup](#Setup)
* [Data](#Data)
    * [Load Data](#Load-Data)
* [Model](#Model)
    * [Load Model](#Load-Model)
    * [Model Evaluation](#Model-Evaluation)
---

# Notebook Summary

This notebook will evaluate the model trained in `02_model_training` on the stanford dog test set.

# Setup

Append parent directory to sys path to be able to import created modules from src directory.

In [1]:
import sys
sys.path.append(os.path.dirname(os.path.abspath("")))

Automatically reload modules when changes are made.

In [2]:
%load_ext autoreload
%autoreload 2

Import libraries and modules.

In [4]:
# Import libraries
import azureml.core
import shutil
import torch
from azureml.core import Dataset, Environment, Experiment, Keyvault, Model, ScriptRunConfig, Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig 
from azureml.train.hyperdrive import BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, RandomParameterSampling
from azureml.train.hyperdrive import choice, uniform
from azureml.widgets import RunDetails
from torchvision import datasets

# Import created modules
from src.utils import load_data

print(f"azureml.core version: {azureml.core.VERSION}")

azureml.core version: 1.20.0


# Data

### Load Data

In [5]:
# Load data
dataloaders, dataset_sizes, class_names = load_data("../data")

# Model

### Load Model

In [ ]:
model_path = "../outputs/model.pt"

model = torch.load(model_path, map_location=lambda storage, loc: storage)
model.eval()

### Model Evaluation

In [ ]:
# Leverage GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Use the test dataloader to load in data in batches
for inputs, labels in dataloaders["test"]:
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    # Make predictions on test data batch
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
    # Add correct predictions from current batch
    running_correct_preds += torch.sum(preds == labels.data)
    
# Calculate the total accuracy over the test set
test_accuracy = running_correct_preds.double() / dataset_sizes["test"]

print(f"Test accuracy: {test_accuracy}")